In [1]:
from src.Graph import *
from src.Node import *
from src.OptimizationAlgorithm import *

In [ ]:
def create_fully_connected_network(layers):
    nodes, parameters = [], []

    input_node = InputNode()
    nodes.append(input_node)

    cur_input_node = input_node
    prev_size = 3
    for i, size in enumerate(layers):
        bias_node = AddBiasNode(parents=[(cur_input_node, 0)])
        parameter_node = ParameterNode(w=np.random.rand(prev_size, size) * 1 - 0.5)
        prod_node = MultiplicationNode(parents=[
            (bias_node, 0),
            (parameter_node, 0)
        ])

        activation_node = TanhNode([(prod_node, 0)]) if i < len(layers) - 1 else SigmoidNode([(prod_node, 0)])
        parameters.append(parameter_node)
        nodes += [bias_node, parameter_node, prod_node, activation_node]
        cur_input_node = activation_node
        prev_size = size + 1
    true_output_node = InputNode()
    cost_node = BinaryCrossEntropyNode([(true_output_node, 0), (cur_input_node, 0)])
    nodes += [true_output_node, cost_node]
    
    return ComputationalGraph(
        nodes=nodes,
        input_nodes=[input_node],
        output_nodes=[cur_input_node],
        true_output_nodes=[true_output_node],
        cost_node=cost_node,
        parameter_nodes=parameters
        )


In [3]:
layers = [4, 4, 1]
graph = create_fully_connected_network(layers)
sgd = GradientDescent(graph.get_parameter_nodes(), learning_rate=0.1)

In [5]:
def boolean_function_dataset(function):
	X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
	Y = np.array([function(x) for x in X])
	return X, Y

def real_function_dataset(length, function, nb_inputs=2, x_min=0, x_max=1):
	X = np.random.rand(length, nb_inputs) * (x_max - x_min) + x_min
	Y = np.array([function(x) for x in X])
	return X, Y

def or_dataset():
	return boolean_function_dataset(lambda x: x[0] or x[1])

def and_dataset():
	return boolean_function_dataset(lambda x: x[0] and x[1])

def xor_dataset():
	return boolean_function_dataset(lambda x: (x[0] and not x[1]) or (not x[0] and x[1]))

def plane_dataset(length, normal_vector=[1, -1]):
	return real_function_dataset(length, lambda x: (np.dot(x, normal_vector) >= 0))

def disk_dataset(length):
	return real_function_dataset(length, lambda x: ((x[0] - 0.5)**2 + (x[1] - 0.5)**2) <= 0.25, x_min = -0.5, x_max=1.5)

In [6]:
X, Y = xor_dataset()

In [11]:
t = []
costs = []
n_passes = 10000

for i in range(n_passes):
    graph.forward(X)
    cost = graph.backward(Y) / X.shape[0]
    sgd.optimize(X.shape[0])

    t.append(i)
    costs.append(cost)
    print(cost)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)